# Agente Iterativo con LM Studio API (qwen3-8b)

Este notebook implementa un agente simple que itera sobre un prompt inicial utilizando la API de LM Studio con el modelo qwen3-8b. El agente está diseñado para razonar y mejorar sus respuestas en cada iteración.


In [1]:
import json
import requests
import time


## Configuración de la API de LM Studio

LM Studio proporciona una API compatible con OpenAI. Configuramos la URL base y los parámetros para conectarnos al modelo qwen3-8b.


In [2]:
# Configuración de la API
API_URL = "http://localhost:1234/v1/chat/completions"  # URL por defecto de LM Studio

# Parámetros para el modelo
MODEL_NAME = "qwen3-8b"  # Nombre del modelo en LM Studio
MAX_TOKENS = 1024        # Máximo de tokens en la respuesta
TEMPERATURE = 0.7        # Temperatura para la generación (0.0-1.0)
TOP_P = 0.9              # Top-p para la generación (0.0-1.0)
MAX_ITERATIONS = 5       # Número máximo de iteraciones del agente


## Función para llamar a la API de LM Studio

Esta función envía una solicitud a la API de LM Studio y devuelve la respuesta generada por el modelo.


In [3]:
def call_lm_studio_api(messages):
    """
    Llama a la API de LM Studio con los mensajes proporcionados.

    Args:
        messages (list): Lista de mensajes en formato de chat para la API.

    Returns:
        str: Texto de la respuesta generada por el modelo.
    """
    headers = {
        "Content-Type": "application/json"
    }

    data = {
        "model": MODEL_NAME,
        "messages": messages,
        "max_tokens": MAX_TOKENS,
        "temperature": TEMPERATURE,
        "top_p": TOP_P,
        "stream": False
    }

    try:
        response = requests.post(API_URL, headers=headers, json=data)
        response.raise_for_status()  # Lanza una excepción si hay un error HTTP

        result = response.json()
        return result["choices"][0]["message"]["content"]
    except requests.exceptions.RequestException as e:
        print(f"Error al llamar a la API: {e}")
        return None
    except (KeyError, IndexError) as e:
        print(f"Error al procesar la respuesta: {e}")
        print(f"Respuesta recibida: {response.text}")
        return None


## Implementación del Agente Iterativo

El agente toma un prompt inicial y lo refina en cada iteración para mejorar la respuesta.


In [4]:
def agente_iterativo(prompt_inicial, max_iteraciones=MAX_ITERATIONS):
    """
    Implementa un agente que itera sobre un prompt inicial para mejorar la respuesta.

    Args:
        prompt_inicial (str): El prompt inicial para el agente.
        max_iteraciones (int): Número máximo de iteraciones.

    Returns:
        dict: Historial de todas las iteraciones y la respuesta final.
    """
    historial = []
    prompt_actual = prompt_inicial

    # Mensaje inicial del sistema para guiar al modelo
    system_message = {
        "role": "system",
        "content": "Eres un asistente útil que no razona. " +
                   "Analiza cuidadosamente cada pregunta y proporciona respuestas detalladas y precisas. " +
                   "Si no estás seguro de algo, indica qué información adicional necesitarías."
    }

    print(f"Iteración 0 (Prompt Inicial): {prompt_inicial}\n")

    for i in range(1, max_iteraciones + 1):
        print(f"\n--- Iteración {i} ---")

        # Preparar mensajes para la API
        messages = [system_message]

        # Añadir historial de conversación si existe
        for entry in historial:
            messages.append({"role": "user", "content": entry["prompt"]})
            messages.append({"role": "assistant", "content": entry["respuesta"]})

        # Añadir el prompt actual
        messages.append({"role": "user", "content": prompt_actual + " /no_think"})

        # Llamar a la API
        print("Generando respuesta...")
        start_time = time.time()
        respuesta = call_lm_studio_api(messages)
        end_time = time.time()

        if respuesta is None:
            print("No se pudo obtener una respuesta. Terminando iteraciones.")
            break

        print(f"Tiempo de respuesta: {end_time - start_time:.2f} segundos")
        print(f"\nRespuesta:\n{respuesta}\n")

        # Guardar esta iteración en el historial
        historial.append({
            "iteracion": i,
            "prompt": prompt_actual,
            "respuesta": respuesta
        })

        # Verificar si debemos continuar iterando
        if i < max_iteraciones:
            # Generar un nuevo prompt para refinar la respuesta
            refinamiento_prompt = (
                "Basándote en tu respuesta anterior, ¿hay algún aspecto que puedas mejorar o profundizar? " +
                "Si crees que tu respuesta es completa, indica 'RESPUESTA FINAL' al principio de tu respuesta. " +
                "De lo contrario, proporciona una respuesta mejorada y más completa."
            )

            messages.append({"role": "assistant", "content": respuesta})
            messages.append({"role": "user", "content": refinamiento_prompt})

            print("Evaluando si la respuesta necesita refinamiento...")
            evaluacion = call_lm_studio_api(messages)

            if evaluacion and evaluacion.startswith("RESPUESTA FINAL"):
                print("\nEl agente considera que ha llegado a una respuesta final.")
                break

            # Actualizar el prompt para la siguiente iteración
            prompt_actual = (
                f"Basándote en el prompt original: '{prompt_inicial}', " +
                f"y tu respuesta anterior, proporciona una respuesta mejorada y más completa."
            )

            print(f"\nPreparando siguiente iteración con nuevo prompt: {prompt_actual}")
        else:
            print("\nSe ha alcanzado el número máximo de iteraciones.")

    # Devolver el historial completo y la respuesta final
    resultado = {
        "prompt_inicial": prompt_inicial,
        "historial": historial,
        "respuesta_final": historial[-1]["respuesta"] if historial else None,
        "iteraciones_totales": len(historial)
    }

    return resultado


## Función para verificar la conexión con LM Studio

Antes de usar el agente, es importante verificar que LM Studio esté ejecutándose y que el modelo qwen3-8b esté cargado.


In [5]:
def verificar_conexion_lm_studio():
    """
    Verifica que LM Studio esté ejecutándose y que se pueda conectar a la API.

    Returns:
        bool: True si la conexión es exitosa, False en caso contrario.
    """
    try:
        # Mensaje simple para probar la conexión
        messages = [
            {"role": "system", "content": "Eres un asistente útil."},
            {"role": "user", "content": "Hola, ¿estás funcionando?"}
        ]

        respuesta = call_lm_studio_api(messages)

        if respuesta:
            print(f"✅ Conexión exitosa con LM Studio. Respuesta de prueba: '{respuesta[:50]}...'")
            return True
        else:
            print("❌ No se pudo obtener una respuesta de LM Studio.")
            return False
    except Exception as e:
        print(f"❌ Error al conectar con LM Studio: {e}")
        print("\nAsegúrate de que:")
        print("1. LM Studio esté ejecutándose en tu computadora")
        print("2. El modelo qwen3-8b esté cargado en LM Studio")
        print("3. El servidor de la API esté activo en http://localhost:1234")
        print("4. No haya un firewall bloqueando la conexión")
        return False


## Instrucciones para usar LM Studio

Antes de ejecutar el agente, asegúrate de seguir estos pasos:

1. Descarga e instala LM Studio desde [https://lmstudio.ai/](https://lmstudio.ai/)
2. Abre LM Studio y descarga el modelo qwen3-8b
3. Carga el modelo en LM Studio
4. Inicia el servidor de la API en LM Studio (generalmente en http://localhost:1234)
5. Ejecuta la celda de verificación de conexión a continuación


In [6]:
# Verificar la conexión con LM Studio antes de usar el agente
conexion_exitosa = verificar_conexion_lm_studio()

if not conexion_exitosa:
    print("\nPor favor, configura LM Studio correctamente antes de continuar.")


✅ Conexión exitosa con LM Studio. Respuesta de prueba: '<think>
Okay, the user asked "Hola, ¿estás funcion...'


## Ejemplo de uso del Agente Iterativo

A continuación, puedes probar el agente con un prompt inicial. El agente iterará sobre el prompt para mejorar la respuesta.


In [ ]:
# Ejemplo de prompt inicial
prompt_ejemplo = "¿Cuáles son las principales causas del cambio climático y qué soluciones existen?"

# Puedes cambiar este prompt por cualquier otro que desees
prompt_usuario = prompt_ejemplo

# Ejecutar el agente iterativo (solo si la conexión fue exitosa)
if conexion_exitosa:
    resultado = agente_iterativo(prompt_usuario)

    print("\n=== Resumen del Proceso ===")
    print(f"Prompt inicial: {resultado['prompt_inicial']}")
    print(f"Número total de iteraciones: {resultado['iteraciones_totales']}")
    print("\n=== Respuesta Final ===\n")
    print(resultado['respuesta_final'])


Iteración 0 (Prompt Inicial): ¿Cuáles son las principales causas del cambio climático y qué soluciones existen?


--- Iteración 1 ---
Generando respuesta...
Tiempo de respuesta: 187.28 segundos

Respuesta:
<think>
Okay, the user is asking about the main causes of climate change and existing solutions. Let me start by recalling what I know.

First, the primary causes are greenhouse gas emissions from human activities like burning fossil fuels for energy, transportation, industry, and deforestation. Also, agriculture contributes through methane from livestock and nitrous oxide from fertilizers. Maybe I should mention land-use changes too, like urbanization affecting natural carbon sinks.

Then, for solutions, renewable energy is a big one—solar, wind, hydropower. Energy efficiency measures in buildings and transportation could help. Reforestation and sustainable agriculture practices like agroforestry or reduced meat consumption might be important. International agreements like the Paris

## Personalización del Agente

Puedes personalizar el comportamiento del agente modificando los siguientes parámetros:


## Función para guardar los resultados

Esta función te permite guardar los resultados del agente para su posterior análisis.


In [ ]:
def guardar_resultados(resultado, nombre_archivo="resultados_agente.json"):
    """
    Guarda los resultados del agente en un archivo JSON.

    Args:
        resultado (dict): Resultado del agente iterativo.
        nombre_archivo (str): Nombre del archivo donde guardar los resultados.
    """
    try:
        with open(nombre_archivo, 'w', encoding='utf-8') as f:
            json.dump(resultado, f, ensure_ascii=False, indent=2)
        print(f"\nResultados guardados exitosamente en '{nombre_archivo}'")
    except Exception as e:
        print(f"\nError al guardar los resultados: {e}")

# Ejemplo de uso:
guardar_resultados(resultado)
